In [2]:
from datetime import datetime,timedelta,date
import pandas as pd
import numpy as np
import seaborn as sns

from __future__ import division
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score
from sklearn.metrics import roc_curve, auc

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import joblib
import matplotlib.pyplot as plt
import plotly.offline as pyoff
import plotly.graph_objs as go

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


# Decision Tree

In [8]:
df = pd.read_csv(r"C:\Users\asus\Documents\Final Project AIP490\Data_Bank\bank-full.csv",sep=';')
convert = {
    "job":{"blue-collar":0,"management":1,"technician":2,"admin.":3,"services":4,"retired":5,"self-employed":6,"entrepreneur":7,"unemployed":8,"housemaid":9,"student":10,"unknown":11},
    "marital":{"married":0,"single":1,"divorced":2},
    "education":{"secondary":0,"tertiary":1,"primary":2,"unknown":3},
    "default":{"no":0,"yes":1},
    "housing":{"no":0,"yes":1},
    "loan":{"no":0,"yes":1},
    "contact":{"cellular":0,"unknown":1,"telephone":2},
    "month":{"may":1,"jul":2,"aug":3,"jun":4,"nov":5,"apr":6,"feb":7,"jan":8,"oct":9,"sep":10,"mar":11,"dec":12},
    "poutcome":{"unknown":0,"failure":1,"other":2,"success":3},
    "y":{"no":0,"yes":1}
    
}
df_done = df.replace(convert)
data_done = df_done
X = data_done.drop('y',axis='columns')
y = data_done['y']

# Tạo pipeline kết hợp oversampling và undersampling
pipeline = Pipeline([
    ('oversample', RandomOverSampler(sampling_strategy=0.4,random_state=42)),  # Oversampling lớp thiểu số để đạt tỷ lệ 1:2
    ('undersample', RandomUnderSampler(sampling_strategy=0.8,random_state=42)),  # Undersampling lớp đa số để đạt tỷ lệ 4:5
])
#  Áp dụng pipeline cho dữ liệu huấn luyện
X_resampled_1, y_resampled_1 = pipeline.fit_resample(X, y)

y_OU = y_resampled_1
X_OU = X_resampled_1
nfolds = 5 #cross validation(3,5,7)

def dtree_grid_search(X_OU,y_OU,nfolds):
    #create a dictionary of all values we want to test
    param_grid = { 'criterion':['gini','entropy'],
                  'max_depth': np.arange(3, 15),
                  'min_samples_split':np.arange(2,7),
                  'min_samples_leaf':np.arange(5,40,5),
                  'max_leaf_nodes': np.arange(3,26)
                 }
    # decision tree model
    dtree_model=DecisionTreeClassifier()
    #use gridsearch to test all values
    dtree_gscv = GridSearchCV(dtree_model, param_grid, cv=nfolds)
    #fit model to data
    dtree_gscv.fit(X_OU, y_OU)
    return dtree_gscv.best_params_

dtree_grid_search(X_OU,y_OU,5)

{'criterion': 'gini',
 'max_depth': 9,
 'max_leaf_nodes': 25,
 'min_samples_leaf': 5,
 'min_samples_split': 2}

In [9]:

X_train, X_test, y_train, y_test =  train_test_split(X_resampled_1,y_resampled_1,test_size = 0.1, random_state= 42)

classifier = DecisionTreeClassifier(criterion='gini',max_depth=9,max_leaf_nodes=25,min_samples_leaf= 5,min_samples_split=2,random_state=42)
classifier = classifier.fit(X_train,y_train)


y_pred = classifier.predict(X_test)
# filename = 'decisionTree_model_1.plk'
# joblib.dump(classifier,filename)
print('Accuracy Score:', accuracy_score(y_test,y_pred))
print('F1 Score:',f1_score(y_test,y_pred))
print('Recall Score: ',recall_score(y_test,y_pred))

Accuracy Score: 0.836348455329808
F1 Score: 0.8109324758842443
Recall Score:  0.8125


# RandomForest

In [2]:
pip install hyperopt

  Obtaining dependency information for hyperopt from https://files.pythonhosted.org/packages/b6/cd/5b3334d39276067f54618ce0d0b48ed69d91352fbf137468c7095170d0e5/hyperopt-0.2.7-py2.py3-none-any.whl.metadata
  Obtaining dependency information for py4j from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.6 MB 960.0 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.6 MB 1.4 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.6 MB 2.0 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.6 MB 2.7 MB/s eta 0:00:01
   ------------------------- -------------- 1.0/1.6 MB 4.6 MB/s eta 0:00:01
   -------------------------------------- - 1.5/1.6 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 5.0 MB/s eta 

In [1]:
import time
import pandas as pd
from hyperopt import hp, fmin, tpe, Trials
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Bắt đầu đếm thời gian
start_time = time.time()

# Load data
df = pd.read_csv(r"C:\Users\asus\Documents\Final Project AIP490\Data_Bank\bank-full.csv",sep=';')

convert = {
    "job": {"blue-collar": 0, "management": 1, "technician": 2, "admin.": 3, "services": 4, "retired": 5,
            "self-employed": 6, "entrepreneur": 7, "unemployed": 8, "housemaid": 9, "student": 10, "unknown": 11},
    "marital": {"married": 0, "single": 1, "divorced": 2},
    "education": {"secondary": 0, "tertiary": 1, "primary": 2, "unknown": 3},
    "default": {"no": 0, "yes": 1},
    "housing": {"no": 0, "yes": 1},
    "loan": {"no": 0, "yes": 1},
    "contact": {"cellular": 0, "unknown": 1, "telephone": 2},
    "month": {"may": 1, "jul": 2, "aug": 3, "jun": 4, "nov": 5, "apr": 6, "feb": 7, "jan": 8, "oct": 9, "sep": 10,
              "mar": 11, "dec": 12},
    "poutcome": {"unknown": 0, "failure": 1, "other": 2, "success": 3},
    "y": {"no": 0, "yes": 1}
}

df_done = df.replace(convert)
X = df_done.drop("y", axis=1)
y = df_done['y']

# Tạo pipeline kết hợp oversampling và undersampling
pipeline = Pipeline([
    ('oversample', RandomOverSampler(sampling_strategy=0.4, random_state=42)),
    ('undersample', RandomUnderSampler(sampling_strategy=0.8, random_state=42)),
])

#  Áp dụng pipeline cho dữ liệu huấn luyện
X_resampled_1, y_resampled_1 = pipeline.fit_resample(X, y)

# Định nghĩa không gian hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 10, 40,1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 20,1),
    'n_estimators': hp.quniform('n_estimators', 100, 900,10),
    'min_samples_split': hp.quniform('min_samples_split', 2, 20,1),
    'max_leaf_nodes': hp.quniform('max_leaf_nodes', 100, 500,10),
    'max_features': hp.choice('max_features', ['sqrt',  'log2']),
    'max_samples': hp.quniform('max_samples', 0.1, 1, 0.1)
}

# Định nghĩa hàm mục tiêu
def objective(params):
    clf = RandomForestClassifier(max_depth=int(params['max_depth']),min_samples_leaf=int(params['min_samples_leaf']),
                                 n_estimators=int(params['n_estimators']),min_samples_split=int(params['min_samples_split']),
                                 max_leaf_nodes=int(params['max_leaf_nodes']),random_state=42, n_jobs=-1, max_features=params['max_features'], max_samples=float(params['max_samples']))
    score = cross_val_score(clf, X_resampled_1, y_resampled_1, cv=5, scoring='f1').mean()
    return -score  # Negative sign for maximization

# Tìm kiếm hyperparameters tốt nhất
trials = Trials()
best = fmin(fn=objective,
            space=param_space,
            algo=tpe.suggest,
            max_evals=150,  # Số lần thử nghiệm tối đa
            trials=trials,
           )

# Kết thúc đếm thời gian
end_time = time.time()

# Tính thời gian chạy chương trình
run_time = end_time - start_time
# Print the best hyperparameters found
print("Best hyperparameters found:")
print(best)
print(run_time)
# Lưu kết quả vào file
# with open('result_rdf.txt', 'w') as f:
#     print("Best Hyperparameters:", best, file=f)
#     print("Runtime:", run_time, "seconds", file=f)

100%|█████████████████████████████████████████████| 150/150 [41:21<00:00, 16.54s/trial, best loss: -0.8799684995355861]
Best hyperparameters found:
{'max_depth': 18.0, 'max_features': 1, 'max_leaf_nodes': 490.0, 'max_samples': 1.0, 'min_samples_leaf': 5.0, 'min_samples_split': 14.0, 'n_estimators': 900.0}
2481.525507926941


In [5]:
def data_label(file):
    df = pd.read_csv(file, sep=';')
    
    convert = {
        "job": {"blue-collar": 0, "management": 1, "technician": 2, "admin.": 3, "services": 4, "retired": 5,
                "self-employed": 6, "entrepreneur": 7, "unemployed": 8, "housemaid": 9, "student": 10,
                "unknown": 11},
        "marital": {"married": 0, "single": 1, "divorced": 2},
        "education": {"secondary": 0, "tertiary": 1, "primary": 2, "unknown": 3},
        "default": {"no": 0, "yes": 1},
        "housing": {"no": 0, "yes": 1},
        "loan": {"no": 0, "yes": 1},
        "contact": {"cellular": 0, "unknown": 1, "telephone": 2},
        "month": {"may": 1, "jul": 2, "aug": 3, "jun": 4, "nov": 5, "apr": 6, "feb": 7, "jan": 8, "oct": 9,
                  "sep": 10, "mar": 11, "dec": 12},
        "poutcome": {"unknown": 0, "failure": 1, "other": 2, "success": 3},
        "y": {"no": 0, "yes": 1}
    }
    df = df.replace(convert)
    return df.values
def random_forest_model(data_array, hold_out_score, n_estimators,
                        min_samples_leaf, n_jobs,
                        max_depth, max_leaf_nodes, min_sample_split, max_sample):
    # intialize X, y
    X = data_array[:, :-1]
    y = data_array[:, -1]

    # imbalance processing
    pipeline = Pipeline([
        ('oversample', RandomOverSampler(sampling_strategy=0.4, random_state=42)),
        # Oversampling lớp thiểu số để đạt tỷ lệ 1:2
        ('undersample', RandomUnderSampler(sampling_strategy=0.8, random_state=42)),
        # Undersampling lớp đa số để đạt tỷ lệ 4:5
    ])

    #  Apply pipeline to dataset train
    X_resampled_1, y_resampled_1 = pipeline.fit_resample(X, y)

    # split data
    X_train, X_test, y_train, y_test = train_test_split(X_resampled_1, y_resampled_1, test_size=hold_out_score,
                                                        random_state=42)

    # Create the Random Forest model
    model = RandomForestClassifier(max_features='sqrt', n_estimators=int(n_estimators), criterion='gini',
                                   min_samples_leaf=int(min_samples_leaf), n_jobs=n_jobs,
                                   oob_score=True, max_depth=max_depth,
                                   max_samples=max_sample, max_leaf_nodes=max_leaf_nodes,
                                   min_samples_split=min_sample_split)

    # Train the model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    # Calculate TP, FP, Pred True, Pred False
    cm = confusion_matrix(y_test, y_pred)
    TP = cm[1, 1]
    FP = cm[0, 1]
    TN = cm[0, 0]
    FN = cm[1, 0]

    # Calculate Precision and Recall
    Precision = precision_score(y_test, y_pred)
    Recall = recall_score(y_test, y_pred)

    # Calculate f1 score and accuracy
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    # tpr npr
    tpr, npr, threshholds = roc_curve(y_test, y_prob[:, 0])
    return model, y_pred, f1, accuracy, TP, FP, TN, FN, Precision, Recall, tpr, npr

#{'max_depth': 18.0, 'max_features': 1, 'max_leaf_nodes': 490.0, 'max_samples': 1.0,
#'min_samples_leaf': 5.0, 'min_samples_split': 14.0, 'n_estimators': 900.0}
#Random Forest Hold-out
if __name__ == "__main__":
    filename = data_label(r"C:\Users\asus\Documents\Final Project AIP490\Data_Bank\bank-full.csv")
    df_arr = np.array(filename)
    df_df = pd.DataFrame(df_arr)
    # Initialize X, y
#     X = df_arr[:, :-1]
#     y = df_arr[:, -1]
    classifier,y_pred, f1, accuracy, TP, FP, TN, FN, Precision, Recall, tpr, npr = random_forest_model(data_array=df_arr,hold_out_score=0.1, n_estimators=900,min_samples_leaf=5.0, n_jobs= -1,max_depth=18,max_leaf_nodes=490,min_sample_split=14,  max_sample=1.0)
    print('Accuracy Score:', accuracy)
    print('F1 Score:',f1)

Accuracy Score: 0.895073754522683
F1 Score: 0.8835341365461847


# XGboost

In [10]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
import xgboost

# Bắt đầu đếm thời gian
start_time = time.time()

df = pd.read_csv(r"C:\Users\asus\Documents\Final Project AIP490\Data_Bank\bank-full.csv",sep=';')

# label y
lbe = LabelEncoder()
df['y'] = lbe.fit_transform(df['y'])
# split data
X = df.drop("y", axis=1).astype('category')
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


# Khai bao space
space={ 'eta': hp.quniform('eta', 0,1,0.001),
        'subsample': hp.quniform('subsample',0,1,0.001),
        'max_depth': hp.quniform('max_depth', 3, 18,1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.uniform('reg_alpha', 0,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        # 'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform('n_estimators', 150, 900, 50),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.4, 1,0.1),
        # 'colsample_bytree': hp.quniform('colsample_bytree', 0,1,0.1),
        # 'colsample_bylevel': hp.quniform('colsample_bylevel', 0,1,0.1),
        # 'colsample_bynode': hp.quniform('colsample_bynode', 0,1,0.1),
        'max_cat_threshold': hp.quniform('max_cat_threshold', 1, 100, 1)
    }
# colsample_bytree=int(space['colsample_bytree']),
def XGboost_train(space):
    clf=xgb(enable_categorical=True, booster='gbtree', device='cuda', tree_method='hist',sampling_method='uniform', scale_pos_weight=float(y_train.value_counts()[0]) / y_train.value_counts()[1],
            updater='grow_gpu_hist', refresh_leaf=1, process_type='default', grow_policy='depthwise', eta=float(space['eta']),subsample=float(space['subsample']),
                    max_depth = int(space['max_depth']),gamma = space['gamma'], reg_alpha = float(space['reg_alpha']), reg_lambda = float(space['reg_lambda']),
                     min_child_weight=int(space['min_child_weight']),n_estimators =int(space['n_estimators']), colsample_bytree=float(space['colsample_bytree']),
                     max_cat_threshold=int(space['max_cat_threshold']), seed=0)

    evaluation = [( X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=20,verbose=0)


    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    f1 = f1_score(y_test, pred>0.5)
    print ("SCORE:", f1)
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()

best_hyperparams = fmin(fn = XGboost_train,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 150,
                        trials = trials, verbose=1)
# Kết thúc đếm thời gian
end_time = time.time()

# Tính thời gian chạy chương trình
run_time = end_time - start_time

print(best_hyperparams)
print(run_time)


  0%|                                                                          | 0/150 [00:00<?, ?trial/s, best loss=?]

C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.


C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.


C:\Users\asus\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning:

[11:31:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:33: You have manually specified the `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.




SCORE:                                                                                                                 
0.39289558665231433                                                                                                    
  1%|▎                                              | 1/150 [00:02<05:54,  2.38s/trial, best loss: -0.7505528527200354]

C:\Users\asus\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning:

[11:31:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.



C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.


C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for bette

SCORE:                                                                                                                 
0.5205811138014528                                                                                                     
  1%|▋                                              | 2/150 [00:02<03:03,  1.24s/trial, best loss: -0.8248562582927909]

...

SCORE:                                                                                                                 
0.43729903536977494                                                                                                    
 99%|████████████████████████████████████████████▍| 148/150 [04:03<00:11,  5.89s/trial, best loss: -0.8942945599292349]

C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.


C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.




SCORE:                                                                                                                 
0.4608695652173913                                                                                                     
 99%|████████████████████████████████████████████▋| 149/150 [04:17<00:08,  8.33s/trial, best loss: -0.8942945599292349]

C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.


C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.




SCORE:                                                                                                                 
0.345703125                                                                                                            
100%|█████████████████████████████████████████████| 150/150 [04:20<00:00,  1.74s/trial, best loss: -0.8942945599292349]
{'colsample_bytree': 1.0, 'eta': 0.487, 'gamma': 7.324767196728767, 'max_cat_threshold': 42.0, 'max_depth': 13.0, 'min_child_weight': 0.0, 'n_estimators': 550.0, 'reg_alpha': 0.9047346246042512, 'reg_lambda': 0.35314609658244456, 'subsample': 0.482}
260.51937532424927


In [13]:
def xgboost_model(data_array, hold_out_score, eta, subsample, max_depth, gamma, reg_alpha, reg_lambda, min_child_weight,
                  n_estimators, colsample_bytree, max_cat_threshold):
        data_df = pd.DataFrame(data_array)
        lbe = LabelEncoder()
        data_df[16] = lbe.fit_transform(data_df[16])
        X = data_df.drop(16, axis=1).astype('category')
        y = data_df[16]
        # imbalance processing
        pipeline = Pipeline([
            ('oversample', RandomOverSampler(sampling_strategy=0.4, random_state=42)),
            # Oversampling lớp thiểu số để đạt tỷ lệ 1:2
            ('undersample', RandomUnderSampler(sampling_strategy=0.8, random_state=42)),
            # Undersampling lớp đa số để đạt tỷ lệ 4:5
        ])
        #  Apply pipeline to dataset train
        X_resampled_1, y_resampled_1 = pipeline.fit_resample(X, y)


        # split data
        X_train, X_test, y_train, y_test = train_test_split(X_resampled_1, y_resampled_1, test_size=hold_out_score, random_state=42)

        clf = xgb(enable_categorical=True, booster='gbtree', tree_method='hist', sampling_method='uniform',
                  scale_pos_weight=float(y_train.value_counts()[0]) / y_train.value_counts()[1], refresh_leaf=1,
                  process_type='default',
                  grow_policy='depthwise', eta=float(eta), subsample=float(subsample),
                  max_depth=int(max_depth), gamma=gamma, reg_alpha=float(reg_alpha), reg_lambda=float(reg_lambda),
                  min_child_weight=int(min_child_weight), n_estimators=int(n_estimators),
                  colsample_bytree=float(colsample_bytree),
                  max_cat_threshold=int(max_cat_threshold), seed=0)

        evaluation = [(X_train, y_train), (X_test, y_test)]

        clf.fit(X_train, y_train,
                eval_set=evaluation, eval_metric="auc",
                early_stopping_rounds=20, verbose=0)

        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)

        # precision and recall
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)

        # tpr npr
        tpr, npr, _ = roc_curve(y_test, y_prob[:, 0])

        # confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Calculate TP, FP, Pred True, Pred False
        TP = cm[1, 1]
        FP = cm[0, 1]
        TN = cm[0, 0]
        FN = cm[1, 0]

        # Return the model, predictions, f1 score, accuracy, and the calculated metrics
        return clf, y_pred, f1_score(y_test, y_pred), accuracy_score(y_test,y_pred),\
               TP, FP, TN, FN, precision, recall, tpr, npr
    
if __name__ == "__main__":
    filename = data_label(r"C:\Users\asus\Documents\Final Project AIP490\Data_Bank\bank-full.csv")
    df_arr = np.array(filename)
    df_df = pd.DataFrame(df_arr)
    # Initialize X, y
    X = df_arr[:, :-1]
    y = df_arr[:, -1]
    
    #XG Boost Hold-out
    classifier, y_pred, f1, accuracy, TP, FP, TN, FN, Precision, Recall, tpr, npr = xgboost_model(data_array=df_arr, hold_out_score=0.1, eta=0.487, subsample=0.482, max_depth=13, gamma=7.324767196728767, reg_alpha=0.9047346246042512, reg_lambda=0.35314609658244456, min_child_weight=0,n_estimators=400, colsample_bytree=1, max_cat_threshold=42.0)
    print('Accuracy Score:', accuracy)
    print('F1 Score:',f1)

C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.

C:\Users\asus\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.



Accuracy Score: 0.9212357361536321
F1 Score: 0.9133231240428791


# Logistic Regression

In [12]:
import time
import pandas as pd
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PowerTransformer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Bắt đầu đếm thời gian
start_time = time.time()

# Load data
df = pd.read_csv(r"C:\Users\asus\Documents\Final Project AIP490\Data_Bank\bank-full.csv", sep=';')

convert = {
    "job": {"blue-collar": 0, "management": 1, "technician": 2, "admin.": 3, "services": 4, "retired": 5,
            "self-employed": 6, "entrepreneur": 7, "unemployed": 8, "housemaid": 9, "student": 10, "unknown": 11},
    "marital": {"married": 0, "single": 1, "divorced": 2},
    "education": {"secondary": 0, "tertiary": 1, "primary": 2, "unknown": 3},
    "default": {"no": 0, "yes": 1},
    "housing": {"no": 0, "yes": 1},
    "loan": {"no": 0, "yes": 1},
    "contact": {"cellular": 0, "unknown": 1, "telephone": 2},
    "month": {"may": 1, "jul": 2, "aug": 3, "jun": 4, "nov": 5, "apr": 6, "feb": 7, "jan": 8, "oct": 9, "sep": 10,
              "mar": 11, "dec": 12},
    "poutcome": {"unknown": 0, "failure": 1, "other": 2, "success": 3},
    "y": {"no": 0, "yes": 1}
}

df_done = df.replace(convert)
# print(df_done.columns)
data_transform = PowerTransformer(method='yeo-johnson')
df_done[['age', 'balance', 'duration']] = data_transform.fit_transform(df_done[['age', 'balance', 'duration']])
X = df_done.drop("y", axis=1)
y = df_done['y']

# Tạo pipeline kết hợp oversampling và undersampling
pipeline = Pipeline([
    ('oversample', RandomOverSampler(sampling_strategy=0.4, random_state=42)),
    ('undersample', RandomUnderSampler(sampling_strategy=0.8, random_state=42)),
])

#  Áp dụng pipeline cho dữ liệu huấn luyện
X_resampled_1, y_resampled_1 = pipeline.fit_resample(X, y)

# Define the objective function
def objective(params):
    model = LogisticRegression(C=float(params['C']), solver=params['solver'], penalty='l2', class_weight=params['class_weight'], max_iter=int(params['max_iter']))
    accuracy = cross_val_score(model, X_resampled_1, y_resampled_1, cv=10).mean()
    return {'loss': -accuracy, 'status': STATUS_OK}

# Define the hyperparameters space
space = {
    'C': hp.choice('C', [0.001, 0.01, 0.1, 1, 10, 100, 200]),
    'solver': hp.choice('solver', ['sag', 'saga']),
    # 'penalty': hp.choice('penalty', ['l1', 'l2']),
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'max_iter': hp.quniform('max_iter', 1, 700, 50)
}

# Run the optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=150, trials=trials)

# Kết thúc đếm thời gian
end_time = time.time()

# Tính thời gian chạy chương trình
run_time = end_time - start_time

# Print the best hyperparameters found
print("Best hyperparameters found:")
print(best)
print(run_time)

  0%|                                                                          | 0/150 [00:00<?, ?trial/s, best loss=?]

C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge




  1%|▎                                             | 1/150 [00:54<2:16:20, 54.90s/trial, best loss: -0.810399048322741]

C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda

  1%|▌                                            | 2/150 [01:50<2:16:05, 55.17s/trial, best loss: -0.8110948534548432]

...

 99%|██████████████████████████████████████████▍| 148/150 [1:44:33<01:42, 51.21s/trial, best loss: -0.8129597221645405]

C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda

 99%|██████████████████████████████████████████▋| 149/150 [1:45:20<00:50, 50.10s/trial, best loss: -0.8129597221645405]

C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge




100%|███████████████████████████████████████████| 150/150 [1:46:15<00:00, 42.50s/trial, best loss: -0.8129597221645405]
Best hyperparameters found:
{'C': 1, 'class_weight': 1, 'max_iter': 550.0, 'solver': 0}
6375.642578840256


In [4]:
def logistic_regression_model(data_array, hold_out_score, C, solver, penalty, class_weight, max_iter):
        # Khởi tạo PowerTransformer với method='yeo-johnson'
        data_transform = PowerTransformer(method='yeo-johnson')
        data_array[:, [0, 5, 11]] = data_transform.fit_transform(data_array[:, [0, 5, 11]])

        # Initialize X, y
        X = data_array[:, :-1]
        y = data_array[:, -1]
        # Chia dữ liệu thành tập train và test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=hold_out_score, random_state=42)

        # Khởi tạo mô hình
        logreg = LogisticRegression(C=float(C), solver=solver, penalty=penalty, class_weight=class_weight,
                                    max_iter=max_iter)

        # Huấn luyện mô hình
        logreg.fit(X_train, y_train)

        # Dự đoán trên tập test
        y_pred = logreg.predict(X_test)

        # Tính toán các giá trị yêu cầu
        f1 = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)

        return logreg, y_pred, f1, accuracy, tp, fp, tn, fn, precision, recall, tpr, fpr

def data_label(file):
    df = pd.read_csv(file, sep=';')
    
    convert = {
        "job": {"blue-collar": 0, "management": 1, "technician": 2, "admin.": 3, "services": 4, "retired": 5,
                "self-employed": 6, "entrepreneur": 7, "unemployed": 8, "housemaid": 9, "student": 10,
                "unknown": 11},
        "marital": {"married": 0, "single": 1, "divorced": 2},
        "education": {"secondary": 0, "tertiary": 1, "primary": 2, "unknown": 3},
        "default": {"no": 0, "yes": 1},
        "housing": {"no": 0, "yes": 1},
        "loan": {"no": 0, "yes": 1},
        "contact": {"cellular": 0, "unknown": 1, "telephone": 2},
        "month": {"may": 1, "jul": 2, "aug": 3, "jun": 4, "nov": 5, "apr": 6, "feb": 7, "jan": 8, "oct": 9,
                  "sep": 10, "mar": 11, "dec": 12},
        "poutcome": {"unknown": 0, "failure": 1, "other": 2, "success": 3},
        "y": {"no": 0, "yes": 1}
    }
    df = df.replace(convert)
    return df.values    
    
if __name__ == "__main__":
    filename = data_label(r"C:\Users\asus\Documents\Final Project AIP490\Data_Bank\bank-full.csv")
    df_arr = np.array(filename)
    df_df = pd.DataFrame(df_arr)
    # Initialize X, y
    X = df_arr[:, :-1]
    y = df_arr[:, -1]    
    
#LR Hold-out
classifier, y_pred, f1, accuracy, TP, FP, TN, FN, Precision, Recall, tpr, npr = logistic_regression_model(data_array=df_arr, hold_out_score=0.1, C=1, solver='sag', penalty='l2', class_weight='balanced', max_iter=550)
    
print('Accuracy Score:', accuracy)
print('F1 Score:',f1)

Accuracy Score: 0.8113666519239274
F1 Score: 0.5111747851002866


C:\Users\asus\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

